In [1]:
!pip list | grep tensorflow

tensorflow                     2.4.1
tensorflow-addons              0.12.1
tensorflow-cloud               0.1.13
tensorflow-datasets            3.0.0
tensorflow-estimator           2.4.0
tensorflow-gcs-config          2.1.7
tensorflow-hub                 0.12.0
tensorflow-metadata            0.29.0
tensorflow-probability         0.12.2


In [2]:
!pip install --upgrade --upgrade-strategy only-if-needed tensorflow-text==2.4.*
!pip install -q tf-models-official==2.4.*

     |████████████████████████████████| 3.4 MB 1.1 MB/s eta 0:00:01


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [4]:
import string
import re
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
from official.nlp.bert import tokenization
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
train_df= pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df= pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample_sub_df= pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [6]:
samples = ['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding',
 "#raining #flooding #Florida #TampaBay #Tampa 18 or 19 days. I've lost count ",
 '#Flood in Bago Myanmar #We arrived Bago',
 'Damage to school bus on 80 in multi car crash #BREAKING ',
 'What a goooooooaaaaaal!!!!!!',
 'this is ridiculous....',
 'What a wonderful day!',
 "No way...I can't eat that shit",
 'Was in NYC last week!',
 'Love my girlfriend',
 'Cooool :)',
 '@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C',
 'We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw']

def clean_tweet(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # remove html
    tweet = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', tweet)        
    # remove special chars 
    # repr to convert to raw string in order to handle this case
    tweet = re.sub(r"\\x\d\d\w\w",'', repr(tweet))
    # remove url
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)

    tweet = re.sub('[%s]' % re.escape(string.punctuation), '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = tweet.lower()
    
    return tweet
    

list(map(clean_tweet, samples))

['our deeds are the reason of this earthquake may allah forgive us all',
 'rockyfire update  california hwy 20 closed in both directions due to lake county fire  cafire wildfires',
 'flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas',
 'im on top of the hill and i can see a fire in the woods',
 'haha south tampa is getting flooded hah wait a second i live in south tampa what am i gonna do what am i gonna do fvck flooding',
 'raining flooding florida tampabay tampa 18 or 19 days ive lost count ',
 'flood in bago myanmar we arrived bago',
 'damage to school bus on 80 in multi car crash breaking ',
 'what a goooooooaaaaaal',
 'this is ridiculous',
 'what a wonderful day',
 'no wayi cant eat that shit',
 'was in nyc last week',
 'love my girlfriend',
 'cooool ',
 'bbcmtd wholesale markets ablaze ',
 'we always try to bring the heavy metal rt ']

In [7]:
def add_loc_keyword(df):
    df.location.fillna('', inplace=True)
    df.keyword.fillna('', inplace=True)
    df.loc[:, 'text_ctx'] = df.location+" "+df.keyword + " " + df.text
    
    return df

add_loc_keyword(train_df)
add_loc_keyword(test_df)

,id,keyword,location,text,text_ctx
0,0,,,Just happened a terrible car crash,Just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s...","Heard about #earthquake is different cities,..."
2,3,,,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese a..."
3,9,,,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...,...
3258,10861,,,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FAS...
3259,10865,,,Storm in RI worse than last hurricane. My city...,Storm in RI worse than last hurricane. My ci...
3260,10868,,,Green Line derailment in Chicago http://t.co/U...,Green Line derailment in Chicago http://t.co...
3261,10874,,,MEG issues Hazardous Weather Outlook (HWO) htt...,MEG issues Hazardous Weather Outlook (HWO) h...


In [8]:
tweets_train_val = train_df.text_ctx.apply(clean_tweet).values.tolist()
tweets_test = test_df.text_ctx.apply(clean_tweet).values.tolist()
labels_train_val = train_df.target.values

from sklearn.model_selection import train_test_split
tweets_train, tweets_val, y_train, y_val = train_test_split(tweets_train_val, labels_train_val, test_size=0.20, random_state=42, shuffle=True)
np.unique(y_train, return_counts=True)[1], np.unique(y_val, return_counts=True)[1]



(array([3468, 2622]), array([874, 649]))

In [9]:
neg, pos = np.bincount(y_train)
total = neg + pos
initial_bias = np.log([pos/neg])
initial_bias, pos/neg


from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights

array([0.87802768, 1.16132723])

## Full bert model using raw classes

In [10]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

def get_pre_process_layer():
    pre_process_tfhub_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
    bert_preprocess_layer = hub.KerasLayer(pre_process_tfhub_url)
    return bert_preprocess_layer
    
    
    
sm_tweets = ["the dog",
            "this is a new 2000 @jk http://www.news.com \U0001F600 !- johndoe@acme.org“) ",
           "#go https://www.ls.ch:80/home?lan=fr long life \x89ÛÒ to sport in 2021 #football HOP LS not me !-)) ?!!!"]

tweets_preprocessed = get_pre_process_layer()(sm_tweets)

print(f'Keys       : {list(tweets_preprocessed.keys())}')
print(f'Shape      : {tweets_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {tweets_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {tweets_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {tweets_preprocessed["input_type_ids"][0, :12]}')


Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (3, 128)
Word Ids   : [ 101 1996 3899  102    0    0    0    0    0    0    0    0]
Input Mask : [1 1 1 1 0 0 0 0 0 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
def get_bert_encoder_layer(trainable=True):
    encoder_tfhub_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
    encoder = hub.KerasLayer(encoder_tfhub_url, trainable=True, name='BERT_encoder')
    return encoder

tweets_encoded = get_bert_encoder_layer(trainable=True)(tweets_preprocessed)

Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7f25f40ed830>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/p

In [13]:
print(tweets_encoded.keys())
tweets_encoded.get('pooled_output')[0]

dict_keys(['sequence_output', 'pooled_output', 'default', 'encoder_outputs'])


<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([ 9.90585923e-01,  9.02491271e-01,  2.33190194e-01,  1.05213933e-01,
       -5.19300282e-01,  8.29919398e-01,  9.98373926e-01, -6.79914653e-01,
       -6.35009408e-01, -9.91357923e-01,  4.23507877e-02, -9.95681524e-01,
       -9.83761191e-01, -9.71929431e-01, -3.39588493e-01,  2.84557790e-01,
        1.38770446e-01,  7.46254772e-02, -7.04332411e-01,  5.19972920e-01,
        1.26761541e-01, -2.72015870e-01,  4.01239395e-01,  6.78625226e-01,
        9.96361077e-01, -5.85017204e-01, -3.80504727e-01,  4.98764142e-02,
       -1.03830164e-02,  6.34233713e-01,  9.20646548e-01,  1.53623939e-01,
        4.59918112e-01, -1.60974070e-01, -1.52723566e-01, -1.32511869e-01,
       -4.70786482e-01, -3.36184621e-01,  1.01673439e-01, -1.28829017e-01,
        2.97841877e-01, -4.44863796e-01,  1.17724992e-01, -7.71950722e-01,
        3.24944556e-02,  3.57039601e-01, -9.96733367e-01,  1.93455428e-01,
        5.14397144e-01, -7.15311766e-01,  9.95311320

In [14]:
X_train = np.array(tweets_train)
X_val = np.array(tweets_val)
X_test = np.array(tweets_test)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices(X_test)

train_ds = train_ds.cache()
val_ds = val_ds.cache()
test_ds = test_ds.cache()


In [15]:
mpl.rcParams['figure.figsize'] = (8, 6)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_metric(history, label, metric, n):
# Use a log scale on y-axis to show the wide range of values.
  plt.plot(history.epoch, history.history[metric],
               color=colors[n], label='Train ' + label)
  plt.plot(history.epoch, history.history['val_'+metric],
               color=colors[n], label='Val ' + label,
               linestyle="--")
  plt.legend()
  plt.xlabel('Epoch')
  plt.ylabel(metric)


In [16]:
THRESHOLD=0.5
METRICS = [        
    tfa.metrics.F1Score(num_classes=2, average='micro', threshold=THRESHOLD)    
]


def build_classifier_model(pre_processing_layer=None, bert_layer=None, max_len=128, metrics=None, optim=None, output_bias=None, 
                           lr=1e-3, do_dropout=False, do_batch_norm=False, dropout=0.5):
    
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')    
    encoder_inputs = pre_processing_layer(text_input)        
    outputs = bert_layer(encoder_inputs)
    clf_output = outputs['pooled_output'] #[:, 0, :]
    out = tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)(clf_output)
    
    model = tf.keras.Model(inputs=text_input, outputs=out)
    model.compile(optim, loss='binary_crossentropy', metrics=metrics)
    
    return model
    
    


resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
# try trainable = true/false    
BATCH_SIZE=64
optim = opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

cbs = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1),
       tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.001,verbose=1),
       tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)]
#with strategy.scope():
model = build_classifier_model(pre_processing_layer=get_pre_process_layer(), bert_layer=get_bert_encoder_layer(trainable=True),
                                      metrics=METRICS, optim=optim, output_bias=initial_bias, lr=1e-3, do_dropout=False, do_batch_norm=False)
history = model.fit(
    train_ds.batch(BATCH_SIZE).prefetch(1), 
    validation_data=val_ds.batch(BATCH_SIZE),
    epochs=20,    
    callbacks=cbs,
    verbose=1)

Epoch 1/20
96/96 [==============================] - 45s 421ms/step - loss: 0.5822 - f1_score: 0.5977 - val_loss: 0.4456 - val_f1_score: 0.7703
Epoch 2/20
96/96 [==============================] - 39s 407ms/step - loss: 0.4339 - f1_score: 0.7638 - val_loss: 0.4255 - val_f1_score: 0.7882
Epoch 3/20
96/96 [==============================] - 39s 406ms/step - loss: 0.3925 - f1_score: 0.7868 - val_loss: 0.4236 - val_f1_score: 0.7884
Epoch 4/20
96/96 [==============================] - 39s 408ms/step - loss: 0.3607 - f1_score: 0.8082 - val_loss: 0.4229 - val_f1_score: 0.7871
Epoch 5/20
96/96 [==============================] - 39s 406ms/step - loss: 0.3260 - f1_score: 0.8338 - val_loss: 0.4293 - val_f1_score: 0.7914
Epoch 6/20
96/96 [==============================] - 39s 411ms/step - loss: 0.2960 - f1_score: 0.8506 - val_loss: 0.4473 - val_f1_score: 0.7888
Epoch 7/20
96/96 [==============================] - 40s 414ms/step - loss: 0.2704 - f1_score: 0.8676 - val_loss: 0.4564 - val_f1_score: 0.7927

In [ ]:
plot_metric(history, "Bert",'loss', 0)
plot_metric(history, "RNN",'f1_score', 1)

In [32]:
test_pred_prob = model.predict(test_ds.batch(64))
threshold = 0.5
test_pred = np.where(test_pred_prob > threshold, 1,0)
test_pred = np.squeeze(test_pred)

In [33]:
zip((test_df.id.values, test_pred))

test_ids = test_df.id.values
test_pred_df = pd.DataFrame({'id':test_ids, 'target':test_pred})
test_pred_df.to_csv("submission.csv", index=False)

In [22]:
do_small_data_fit = True

if do_small_data_fit:
    def create_small_data(X,y, nb_per_class=1):
        true_idxs = np.where(y == True)[0][:nb_per_class]
        false_idxs = np.where(y == False)[0][:nb_per_class]        
        idxs = np.concatenate((true_idxs, false_idxs)).tolist()
        X_sm = X[idxs]
        y_sm = y[idxs]
        
        return X_sm, y_sm

    # try with 2 and 10 samples
    SM_NB=10
    X_train_sm, y_train_sm = create_small_data(X_train, y_train, nb_per_class=int(SM_NB/2))
    
    optim = opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
    sm_model = build_classifier_model(pre_processing_layer=get_pre_process_layer(), bert_layer=get_bert_encoder_layer(trainable=True),
                                      metrics=METRICS, optim=optim, output_bias=initial_bias, lr=1e-3, do_dropout=False, do_batch_norm=False)
    sm_history = sm_model.fit(
        X_train_sm,
        y_train_sm,
        batch_size=SM_NB,
        epochs=50,
        verbose=0)

    threshold = 0.5
    sm_pred_prob = sm_model.predict(X_train_sm)
    sm_pred = np.where(sm_pred_prob > threshold, 1,0)
    print("pred probabilities: ")
    print(sm_pred_prob)
    print("pred:")
    print(sm_pred)

pred probabilities: 
[[0.9973653 ]
 [0.99668175]
 [0.99776864]
 [0.99774987]
 [0.99655664]
 [0.00431994]
 [0.00288992]
 [0.00506494]
 [0.00507691]
 [0.00284994]]
pred:
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [ ]:
sample_tokens.shape, sample_masks, sample_ids

In [ ]:

max_len = SEQ_LEN
input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [ ]:
bert_layer([sample_tokens[:], sample_masks[:], sample_ids[:]])

## Small bert

In [ ]:
THRESHOLD=0.5
METRICS = [        
    tfa.metrics.F1Score(num_classes=2, average='micro', threshold=THRESHOLD)    
]


def build_classifier_model(metrics=None, optim=None, output_bias=None, lr=1e-3, do_dropout=False, do_batch_norm=False, dropout=0.5):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    x = outputs['pooled_output']    
    #x = tf.keras.layers.Dropout(0.1)(x)
    
    # also include batchnorm
    #tf.keras.layers.Dense(120),
    #    tf.keras.layers.BatchNormalization(),
    #    tf.keras.layers.Activation('sigmoid'),
    
    # try different initializers

    # clean texts
    
    x = tf.keras.layers.Dense(256)(x)
    if do_batch_norm is True:
        x = tf.keras.layers.BatchNormalization()(x)
    if do_dropout is True:
        x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.activations.relu(x)
    
    '''
    x = tf.keras.layers.Dense(128)(x)
    if do_batch_norm is True:
        x = tf.keras.layers.BatchNormalization()(x)
    if do_dropout is True:
        x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.activations.relu(x)        
    
    x = tf.keras.layers.Dense(64)(x)
    if do_batch_norm is True:
        x = tf.keras.layers.BatchNormalization()(x)
    if do_dropout is True:
        x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.activations.relu(x)
    '''
    x = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(x)
    model =  tf.keras.Model(text_input, x)
    if optim is None:
        optim = tf.keras.optimizers.Adam(lr=lr)
        
    model.compile(optimizer= optim,
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    
    return model



### train model on full data

In [ ]:
EPOCH_NB=60

steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * EPOCH_NB
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
adamw = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')



In [ ]:

BATCH_SIZE = 64

cbs = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1),
      tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.001,verbose=1)]

model = build_classifier_model(metrics=METRICS, optim=adamw, output_bias=initial_bias, lr=3e-5, do_dropout=True, do_batch_norm=False)

train_ds = train_ds.batch(BATCH_SIZE)
val_ds = val_ds.batch(BATCH_SIZE)

history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=EPOCH_NB,
                    callbacks=cbs)

### Test prediction


In [ ]:
#!rm submission.csv
#!ls -la

### Test model on small dataset

In [ ]:
do_small_data_fit = True

if do_small_data_fit:
    def create_small_data(X,y, nb_per_class=1):
        true_idxs = np.where(y == True)[0][:nb_per_class]
        false_idxs = np.where(y == False)[0][:nb_per_class]        
        idxs = np.concatenate((true_idxs, false_idxs)).tolist()           
        X_sm = X[idxs]
        y_sm = y[idxs]
        return X_sm, y_sm

    # try with 2 and 10 samples
    SM_NB=10
    X_train_sm, y_train_sm = create_small_data(X_train, y_train, nb_per_class=int(SM_NB/2))


    sm_model = build_classifier_model(metrics=METRICS, optim=None, output_bias=initial_bias, lr=3e-5, do_dropout=False, do_batch_norm=False)
    sm_history = sm_model.fit(
        X_train_sm,
        y_train_sm,
        batch_size=SM_NB,
        epochs=50,
        verbose=0)

    threshold = 0.5
    sm_pred_prob = sm_model.predict(X_train_sm)
    sm_pred = np.where(sm_pred_prob > threshold, 1,0)
    print("pred probabilities: ")
    print(sm_pred_prob)
    print("pred:")
    print(sm_pred)

# old

In [9]:
module_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
#"https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
#'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_layer = hub.KerasLayer(module_url, trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
bert_tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [10]:
def encode_for_bert(texts, tokenizer, max_len=128):
    
    tokens = []
    masks = []
    segments = []
    
    
    for text in texts:
        text_tokens = tokenizer.tokenize(text)

        text_tokens = text_tokens[:max_len-2]
        input_sequence = ["[CLS]"] + text_tokens + ["[SEP]"]
        pad_len = max_len - len(input_sequence)

        token_seq = tokenizer.convert_tokens_to_ids(input_sequence)
        token_seq += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        tokens.append(token_seq)
        masks.append(pad_masks)
        segments.append(segment_ids)
        

    return np.array(tokens), np.array(masks), np.array(segments)
    
    
    
SEQ_LEN = 100
sample_tokens, sample_masks, sample_ids = encode_for_bert(samples[:2], bert_tokenizer, max_len=SEQ_LEN)


In [9]:
module_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
#"https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
#'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_layer = hub.KerasLayer(module_url, trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
bert_tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [10]:
def encode_for_bert(texts, tokenizer, max_len=128):
    
    tokens = []
    masks = []
    segments = []
    
    
    for text in texts:
        text_tokens = tokenizer.tokenize(text)

        text_tokens = text_tokens[:max_len-2]
        input_sequence = ["[CLS]"] + text_tokens + ["[SEP]"]
        pad_len = max_len - len(input_sequence)

        token_seq = tokenizer.convert_tokens_to_ids(input_sequence)
        token_seq += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        tokens.append(token_seq)
        masks.append(pad_masks)
        segments.append(segment_ids)
        

    return np.array(tokens), np.array(masks), np.array(segments)
    
    
    
SEQ_LEN = 100
sample_tokens, sample_masks, sample_ids = encode_for_bert(samples[:2], bert_tokenizer, max_len=SEQ_LEN)
